Problem 4

In [40]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [41]:
hotel_prob = pd.read_csv('hotel_prob.csv')

In [42]:
def pricing(r, din, dout):
    if r == 'Q':
        base = 200
    elif r == 'K':
        base = 250
    else:
        base = 300
    price = 0
    for d in range(din, dout):
        if d in [4, 5, 6]:
            price += base * 1.15
        else:
            price += base
    return price

In [43]:
# Part 1a

prob = hotel_prob.loc[hotel_prob['r'] == 'Q', 'probability'].sum()
print(f'The probability that the hotel receives a reservation for a Queen room (with any check-in / check-out dates) in a given (4 hour) period is {prob:.2f}')

The probability that the hotel receives a reservation for a Queen room (with any check-in / check-out dates) in a given (4 hour) period is 0.32


In [44]:
# Part 1b

prob = hotel_prob.loc[(hotel_prob['din'] == 4) & (hotel_prob['dout'] == 7), 'probability'].sum()
print(f'The probability that the hotel receives a reservation that checks in on Thursday 9/14/2023 and checks out on Sunday 9/17/2023 (with any kind of room) in a given (4 hour) period is {prob:.2f}')

The probability that the hotel receives a reservation that checks in on Thursday 9/14/2023 and checks out on Sunday 9/17/2023 (with any kind of room) in a given (4 hour) period is 0.01


In [45]:
# Part 1c

prob = 1 - hotel_prob['probability'].sum()
print(f'The probability that there is no request in a given (4 hour) period is {prob:.2f}')

The probability that there is no request in a given (4 hour) period is 0.26


In [46]:
# Part 1d

T = 540
expval = hotel_prob.loc[(hotel_prob['r'] == 'C') & (hotel_prob['din'] == 5) & (hotel_prob['dout'] == 6), 'probability'].sum() * T
print(f'The expected value of the number of requests for a California King room, with check-in on Friday 9/15/2023 (d = 5) and check-out on Saturday 9/16/2023 (d = 6) is {expval:.2f}')

The expected value of the number of requests for a California King room, with check-in on Friday 9/15/2023 (d = 5) and check-out on Saturday 9/16/2023 (d = 6) is 6.00


Part 2a

\begin{align}

\text{maximize}\quad & \sum_{i=1}^N r_i x_i \;\;\;\; \text{[Objective Function]}\\
\text{subject to} \quad & \sum_{i=1}^{N} x_i \cdot \mathbb{1}_{(\ell \in \text{itineraries}_i) \wedge (r \geq \text{din}_i) \wedge (r < \text{dout}_i)} \leq B_{\ell} \quad \forall d \in \{1,\dots, 7\}\;\;\;\; \text{[Everyday Occupancy Constraint]}\\
& x_i \geq 0, \quad \forall i \in \{1,\dots, N\}. \;\;\;\; \text{[Non-Negativity Constraint]}


\end{align}

where decision variables are given as follows:

$r_{i}$ is the revenue from accepting a request of type $i$

$x_{i}$ is the number of requests of type $i$ that are accepted

$B_{r}$ is the total capacity of room type $r$

to maximize the total revenue.

In [47]:
# Part 2b

import numpy as np

nItineraries = hotel_prob.shape[0]
nLegs = ['Q', 'K', 'C']
nDays = [1, 2, 3, 4, 5, 6, 7]

# Let's assume we have the following seats on the legs:
B = np.array([50, 50, 20])

# Assume we are selling over 750 periods. (Again, can think
# of these as days or smaller periods than days, e.g., 12hr/6hr/2hr periods.)
T = 540

# Below is a list of lists. Each element is a list
# that specifies which legs are used in the itinerary. We'll
# use this when we define our constraints for the LP momentarily.
itineraries_to_legs = hotel_prob[['r', 'din', 'dout']].drop_duplicates()
itineraries_to_legs = [list(itineraries) for itineraries in itineraries_to_legs.values]

revenue = np.array(hotel_prob.apply(lambda x: pricing(x['r'], x['din'], x['dout']), axis=1))
probability = np.array(hotel_prob.probability)

forecast = T * probability

# Formulate the LP:
from gurobipy import * 

# Create the model and the decision variables.
m = Model()
m.Params.LogToConsole = 0

x = m.addVars(nItineraries, lb = 0, ub = forecast)

# Define the constraints.
# Notice how the itineraries_to_legs list is used to define the constraint;
# for each leg ell, only add up those x[i]'s for which the itinerary i uses leg ell. 
leg_capacity_constrs = {}
for ell in nLegs:
    for day in nDays:
        leg_capacity_constrs[(nLegs.index(ell), nDays.index(day))] = m.addConstr(sum(x[i] for i in range(nItineraries) if ((ell in itineraries_to_legs[i]) and (day >= itineraries_to_legs[i][1]) and (day < itineraries_to_legs[i][2]))) <= B[nLegs.index(ell)])

# Specify the objective
m.setObjective( sum(revenue[i] * x[i] for i in range(nItineraries)), GRB.MAXIMIZE)

# Solve 
m.update()
m.optimize()

# Save the LP objective
LP_obj = m.objval

# Display the static allocation
# print( [x[i].x for i in range(nItineraries)])
# print( forecast)
print('The optimal revenue is: ', LP_obj)

# for ell in nLegs:
#     for day in nDays:
#         print(f'ell: {ell}, day: {day}, booking: {leg_capacity_constrs[(nLegs.index(ell), nDays.index(day))].slack}')

# for i in itineraries_to_legs:
#     print(f'{i}: {x[itineraries_to_legs.index(i)].x}')

The optimal revenue is:  172140.0017788


In [48]:
# Part 2c

top5 = sorted(range(len(x)), key = lambda i: x[i].x, reverse=True)[:5]
for i in top5:
    print(f'{itineraries_to_legs[i]}: {x[i].x}')

['K', 2, 3]: 19.99999814000001
['K', 3, 4]: 15.999997820000011
['K', 4, 5]: 13.999998200000011
['Q', 1, 2]: 10.00000026
['K', 2, 4]: 10.00000026


In [49]:
# Part 2d

for ell in nLegs:
    for day in nDays:
        print(f'RoomType: {ell}, Day: {day}, DualVariable: {leg_capacity_constrs[(nLegs.index(ell), nDays.index(day))].pi}')

RoomType: Q, Day: 1, DualVariable: 0.0
RoomType: Q, Day: 2, DualVariable: 200.0
RoomType: Q, Day: 3, DualVariable: 200.0
RoomType: Q, Day: 4, DualVariable: 230.0
RoomType: Q, Day: 5, DualVariable: 229.99999999999997
RoomType: Q, Day: 6, DualVariable: 0.0
RoomType: Q, Day: 7, DualVariable: 0.0
RoomType: K, Day: 1, DualVariable: 0.0
RoomType: K, Day: 2, DualVariable: 250.0
RoomType: K, Day: 3, DualVariable: 250.0
RoomType: K, Day: 4, DualVariable: 287.5
RoomType: K, Day: 5, DualVariable: 0.0
RoomType: K, Day: 6, DualVariable: 0.0
RoomType: K, Day: 7, DualVariable: 0.0
RoomType: C, Day: 1, DualVariable: 0.0
RoomType: C, Day: 2, DualVariable: 300.0
RoomType: C, Day: 3, DualVariable: 300.0
RoomType: C, Day: 4, DualVariable: 345.0
RoomType: C, Day: 5, DualVariable: 345.0
RoomType: C, Day: 6, DualVariable: 345.0
RoomType: C, Day: 7, DualVariable: 0.0


In [50]:
# Part2e

delta = 10 * (sum([leg_capacity_constrs[(0, i)].pi for i in range(7)]) - sum([leg_capacity_constrs[(1, i)].pi for i in range(7)]))

print(f'The predicted change in revenue is ${delta:.2f}')

The predicted change in revenue is $725.00


In [51]:
# Part 3

np.random.seed(50)
T = 540
random_sequences = np.random.choice(range(nItineraries + 1), size = (100, T), p = np.append(probability, 1 - probability.sum()))

In [52]:
# Part 3a

nSimulations = 100

results_myopic_revenue = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    b = np.tile(b, (7, 1)).T
    arrival_sequence = random_sequences[s]
    
    for t in range(T):
        # Stop if all seats have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]

        if (i < nItineraries):        

            r = itineraries_to_legs[i][0]
            din = itineraries_to_legs[i][1]
            dout = itineraries_to_legs[i][2]
            
            if dout != 8:
                # If there is a free seat on each leg for this itinerary...
                if ((b[nLegs.index(r)][nDays.index(din):nDays.index(dout)] > 0).all()):
                    # ... accept the request!
                    b[nLegs.index(r)][nDays.index(din):nDays.index(dout)] -= 1
                    total_revenue += revenue[i]
            else:
                # If there is a free seat on each leg for this itinerary...
                if ((b[nLegs.index(r)][nDays.index(din):] > 0).all()):
                    # ... accept the request!
                    b[nLegs.index(r)][nDays.index(din):] -= 1
                    total_revenue += revenue[i]

    
    results_myopic_revenue[s] = total_revenue

print("Mean myopic revenue: ", results_myopic_revenue.mean())

Mean myopic revenue:  147472.025


In [53]:
# Part 3b

def opportunity_cost(r, din, dout):
    if dout != 8:
        return sum([leg_capacity_constrs[(nLegs.index(r), i)].pi for i in range(nDays.index(din), nDays.index(dout))])
    else:
        return sum([leg_capacity_constrs[(nLegs.index(r), i)].pi for i in range(nDays.index(din), 7)])
    
opportunity_cost('K', 2, 5)

787.5

In [54]:
# Part 3c

def bpc(b, t, r, din, dout):
    for ell in nLegs:
        for day in nDays:
            leg_capacity_constrs[(nLegs.index(ell), nDays.index(day))].rhs = b[nLegs.index(ell), nDays.index(day)]

    for i in range(nItineraries):
        x[i].ub = (T - t) * probability[i]
    
    m.update()
    m.optimize()
    
    dual_val = opportunity_cost(r, din, dout)
    
    return dual_val

nSimulations = 100

results_revenue = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    b = np.tile(b, (7, 1)).T
    # add a dummy itinerary to the end of the sequence to ensure that the last request is always rejected
    # b = np.append(b, np.zeros((3, 1)), axis = 1)

    arrival_sequence = random_sequences[s]
    
    for t in range(T):
        # Stop if all seats have been sold:
        if ((b == 0).all()):
            break
        
        i = arrival_sequence[t]

        if (i < nItineraries):        

            r = itineraries_to_legs[i][0]
            din = itineraries_to_legs[i][1]
            dout = itineraries_to_legs[i][2]

            total_bid_price = bpc(b, t, r, din, dout)

            if dout != 8:
                # If there is a free seat on each leg for this itinerary...
                if ((revenue[i] >= total_bid_price) & (b[nLegs.index(r)][nDays.index(din):nDays.index(dout)] > 0).all()):
                    # ... accept the request!
                    b[nLegs.index(r)][nDays.index(din):nDays.index(dout)] -= 1
                    total_revenue += revenue[i]
            else:
                # If there is a free seat on each leg for this itinerary...
                if ((revenue[i] >= total_bid_price) & (b[nLegs.index(r)][nDays.index(din):] > 0).all()):
                    # ... accept the request!
                    b[nLegs.index(r)][nDays.index(din):] -= 1
                    total_revenue += revenue[i]
    
    results_revenue[s] = total_revenue

print("Mean revenue: ", results_revenue.mean())

Mean revenue:  163673.975


<!-- a) What is the probability that the hotel receives a reservation for a Queen room (with any check-in / check-out dates) in a given (4 hour) period? -->

